<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_services_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases a novel application of Qdrant vector DB in the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG) system. Given a user query, we retrieve the corresponding OpenAPI specification from Qdrant vector DB. The retrieved service specification is then seamlessly injected into the Haystack pipeline, enabling the dynamic invocation of selected API services. This innovative approach, integrating any OpenAPI spec service with LLMs, opens a new era for RAG systems to generate contextually rich responses from any openapi service.

## 1. Setup
This notebook demos Haystack 2.x service based RAG, using two openapi services: GitHub's compare_branches and SerperDev search API.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [ ]:
!pip install -q "grpcio-tools==1.42" sentence-transformers openapi3 jsonref qdrant-haystack git+https://github.com/deepset-ai/haystack.git

In [ ]:
import getpass
import os
import json
import requests
from typing import List, Dict, Any
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever

from haystack import Pipeline, Document
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions, OutputAdapter
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.others import Multiplexer
from haystack.dataclasses import ChatMessage, ByteStream

### 1.1 Let's collect the API keys for LLM provider, serper.dev, and github token

In [ ]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")
serper_dev_key = getpass.getpass("Enter serperdev api key:")
github_token = getpass.getpass("Enter your github token:")

## 2. Setup QdrantDocumentStore

QdrantDocumentStore is one of many vector DB integrations we offer via Haystack Core Integrations [project](https://github.com/deepset-ai/haystack-core-integrations/)

In [ ]:
document_store = QdrantDocumentStore(
    path="./qdrant_101",
    index="Document",
    embedding_dim=768,
    recreate_index=True
  )

### 2.1 Add some helper functions we'll use in both indexing and retrieval pipelines

In [ ]:
def prepare_fc(openai_functions_schema: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "tools": [{
            "type": "function",
            "function": openai_functions_schema
        }],
        "tool_choice": {
            "type": "function",
            "function": {"name": openai_functions_schema["name"]}
        }
    }

def create_docs(functions, specs, system_messages, service_credentials):
  docs = []
  for function, spec, system_message, service_credential in zip(functions, specs, system_messages, service_credentials):
    d = Document(content=json.dumps(function),
                 meta={"spec": json.dumps(spec),
                       "system_message": system_message,
                       "service_credential": service_credential})
    docs.append(d)
  return docs

## 3. Index two OpenAPI specs: Github (compare_branches) and [SerperDev](https://serper.dev)

Each indexed service Document contains the following descriptors:

* OpenAI function calling JSON
* OpenAPI specification
* System message for the service
* Service credentials

In [ ]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("fetcher", LinkContentFetcher())
indexing_pipeline.add_component("mx", Multiplexer(List[ByteStream]))
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-mpnet-base-v2",
                                                                                 meta_fields_to_embed=["spec", "system_message", "service_credential"]))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.add_component("a1", OutputAdapter("{{functions | create_docs(specs, system_messages, service_credentials)}}",
                                                     List[Document],
                                                      {"create_docs": create_docs}))

indexing_pipeline.add_component("a2", OutputAdapter("{{sources | json_objects}}",
                                                     List[Dict[str, Any]],
                                                      {"json_objects": lambda sources: [json.loads(s.to_string()) for s in sources]}))

indexing_pipeline.connect("fetcher.streams", "mx")
indexing_pipeline.connect("mx", "spec_to_functions.sources")
indexing_pipeline.connect("mx", "a2.sources")
indexing_pipeline.connect("a2", "a1.specs")
indexing_pipeline.connect("spec_to_functions.functions", "a1")
indexing_pipeline.connect("a1.output", "embedder.documents")
indexing_pipeline.connect("embedder", "writer")

In [ ]:
result = indexing_pipeline.run(data={"fetcher": {"urls":["https://bit.ly/github_compare","https://bit.ly/serper_dev_spec"]},
                                     "a1":{"service_credentials": [github_token, serper_dev_key],
                                           "system_messages": [requests.get("https://bit.ly/pr_auto_system").text,
                                                               requests.get("https://bit.ly/serper_dev_system_prompt").text]
                                           }
                                     })
print(f"\nWrote {result['writer']['documents_written']} documents")

## 4. Build the retrieval/LLM generation pipeline

In [ ]:
from haystack.utils import Secret

pipe = Pipeline()
pipe.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-mpnet-base-v2"))
pipe.add_component("retriever", QdrantEmbeddingRetriever(document_store=document_store))
pipe.add_component("mx", Multiplexer(List[Document]))
pipe.add_component("a1", OutputAdapter("{{documents[0].content | tojson | prepare_fc}}", Dict[str, Any], {"prepare_fc": prepare_fc, "tojson": lambda s: json.loads(s)}))
pipe.add_component("a2", OutputAdapter("{{documents[0].meta['spec'] | tojson}}", Dict[str, Any], {"tojson": lambda s: json.loads(s)}))
pipe.add_component("a3", OutputAdapter("{{documents[0].meta['service_credential']}}", str))
pipe.add_component("a4", OutputAdapter("{{documents[0].meta['system_message'] | cm}}", List[ChatMessage], {"cm": lambda s: [ChatMessage.from_system(s)]}))
pipe.add_component("a5", OutputAdapter("{{system_message + service_response}}", List[ChatMessage], {"cm": lambda s: [ChatMessage.from_system(s)]}))
pipe.add_component("fn_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-3.5-turbo-0613"))
pipe.add_component("openapi_container", OpenAPIServiceConnector())
pipe.add_component("llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-4-1106-preview", streaming_callback=print_streaming_chunk))

pipe.connect("embedder", "retriever")
pipe.connect("retriever.documents", "mx")
pipe.connect("mx", "a1.documents")
pipe.connect("mx", "a2.documents")
pipe.connect("mx", "a3.documents")
pipe.connect("mx", "a4.documents")
pipe.connect("a1", "fn_llm.generation_kwargs")
pipe.connect("a2", "openapi_container.service_openapi_spec")
pipe.connect("a3", "openapi_container.service_credentials")
pipe.connect("a4", "a5.system_message")
pipe.connect("a5", "llm.messages")
pipe.connect("openapi_container.service_response", "a5.service_response")
pipe.connect("fn_llm.replies", "openapi_container.messages")

## 5. Retrieval, service invocation and LLM generation

Here is where everything gets interesting! Based on the prompt below, we'll retrieve the top_k=1 document from QdrantDocumentStore. That'll be the relevant service Document. We'll then proceed to use service descriptors from the retrieved Document to initiate:

* Based on the prompt, we'll use function calling to resolve the parameters for service invocation.
* We'll invoke the OpenAPI-compliant service with those parameters.
* The response from the service will be joined with the service prompt for that service.
* The final LLM response will be generated.


This process involves straightforward intent recognition that utilizes vector similarity to accurately identify and retrieve the appropriate service Document. Theoretically, it's possible to have dozens of services indexed within the QdrantDocumentStore. Once we retrieve the right service Documents, we will rely on the Haystack pipeline to perform the magic and observe the response generated by the LLM.

In [ ]:
# Select on of these two prompts and run the pipeline
user_prompt = "Search web and tell me why was Sam Altman ousted from OpenAI"
user_prompt = "Compare branches main and test/benchmarks2.0, in project deepset-ai, repo haystack"

result = pipe.run(data={"fn_llm": {"messages":[ChatMessage.from_user(user_prompt)]}, "embedder": {"text": user_prompt}})

## 6. Obligatory pretty print version

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result["llm"]["replies"][0].content))


## Thank you, questions?

<a href="www.qr-code-generator.com/" border="0" style="cursor:default" rel="nofollow"><img src="https://chart.googleapis.com/chart?cht=qr&chl=https%3A%2F%2Fgithub.com%2Fvblagoje%2Fnotebooks%2Fblob%2Fmain%2Fhaystack2x-demos%2Fhaystack_rag_services_demo.ipynb&chs=180x180&choe=UTF-8&chld=L|2"></a>

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://haystack.deepset.ai/advent-of-haystack
- https://x.com/vladblagoje